In [ ]:
# Imports

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [17]:
# Neural Net Class definition

In [18]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # 2 convolution layers
        # 1 input image channel, 6 output channels, 5x5 convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # 3 fully connected layers
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        # Convolution layers
        # x is the input image to the Neural Network
        # Max pooling over a 2x2 window (since it is square, it can also be specified as 2, instead of (2,2))
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # Now applying flattening before passing it to the Fully Connected layers
        # flatten all dimensions except the batch dimension
        x = torch.flatten(x, 1)
        
        # Now the Fully Connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [19]:
# Printing the Neural Net object
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [20]:
# Learnable parameters of models
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's weight matrix dimensions

10
torch.Size([6, 1, 5, 5])


In [21]:
# A random 32x32 image as input
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0840, -0.0838,  0.0772,  0.0769, -0.1003, -0.0323, -0.0429,  0.0671,
         -0.0304,  0.0753]], grad_fn=<AddmmBackward0>)


In [22]:
# Zero the gradient buffers of all parameters and backprops with random gradients
net.zero_grad()
out.backward(torch.randn(1,10))

# Calculating loss using Mean Squared Error (MSE)
output = net(input)
target = torch.randn(10) # a dummy target
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(0.6828, grad_fn=<MseLossBackward0>)


In [23]:
# Following the grad_fn to trace back the computation graph
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [24]:
# Back Propagation

In [25]:
# Zero the gradient buffers and then compare conv1's bias gradients before and after the backpropagation
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0077,  0.0071,  0.0002, -0.0154, -0.0118, -0.0091])


In [26]:
# Update the weights and biases

# Create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# In your training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # does the update